一个简单的图：
![graph](graph1.png)

### 从节点、特征和度来刻画一个简单的图

In [12]:
import numpy as np

In [13]:
# feature matrix
X = np.array([[2, 2, 2],   # 1
              [1, 1, 1],   # 2
              [4, 4, 4],   # 3
              [3, 3, 3]])  # 4
X

array([[2, 2, 2],
       [1, 1, 1],
       [4, 4, 4],
       [3, 3, 3]])

In [14]:
# adjacent matrix
A = np.array([[0, 1, 0, 1],
              [1, 0, 0, 1],
              [0, 0, 0, 1],
              [1, 1, 1, 0]])
A

array([[0, 1, 0, 1],
       [1, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 1, 1, 0]])

In [15]:
# degree matrix
D = np.diag(np.sum(A, axis=1))
D

array([[2, 0, 0, 0],
       [0, 2, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 3]])

### 最简单的图神经网络
目的：不断聚合邻居的信息来更新自身

做法：将邻接矩阵跟结点特征矩阵相乘

公式表达：$\mathbf{H}^{k+1}=\mathbf{A}\mathbf{H}^{k}$

注意：$\mathbf{H}^{0}= \mathbf{X}$

验证：此方式做到的结果是用结点的邻居结点简单相加得到新的表示

In [16]:
print('经过一次迭代得到的结果')
print(A.dot(X))

经过一次迭代得到的结果
[[4 4 4]
 [5 5 5]
 [3 3 3]
 [7 7 7]]


#### 以结点1为例验证更新后的结点1为结点2和结点4相加得到（2，4为其邻居）

In [19]:
print(X[1] + X[3])
print((X[1] + X[3]) == A.dot(X)[0])

[4 4 4]
[ True  True  True]


### 简单的神经网络进阶1
简单图神经网络总结：每一次迭代都用邻居结点更新自身

注意：每一次迭代只是变化了结点信息，图的结构并未改变，即只更新$\mathbf{X}$,$\mathbf{A}$和$\mathbf{D}$并未发生任何变化

优点：考虑到了邻居信息

缺点：丢失了自身信息

解决问题：引入自身结点信息

公式表达：
$$\tilde{\mathbf{A}}=\mathbf{A}+\mathbf{I}_{N}$$
$$\mathbf{H}^{k+1}=\tilde{\mathbf{A}}\mathbf{H}^{k}$$

In [21]:
I = np.diag(np.ones(4))
hat_a = I + A
hat_a

array([[1., 1., 0., 1.],
       [1., 1., 0., 1.],
       [0., 0., 1., 1.],
       [1., 1., 1., 1.]])

In [22]:
print('考虑自身信息后经过一次迭代得到的结果：')
print(hat_a.dot(X))

考虑自身信息后经过一次迭代得到的结果：
[[ 6.  6.  6.]
 [ 6.  6.  6.]
 [ 7.  7.  7.]
 [10. 10. 10.]]


#### 以结点1为例验证更新后的结点1为结点2和结点4和自身相加得到（2，4为其邻居 + 自身）

In [23]:
print(X[1] + X[3] + X[0])
print((X[1] + X[3] + X[0]) == hat_a.dot(X)[0])

[6 6 6]
[ True  True  True]


#### 小结
目前做到的效果：即考虑到了邻居结点的信息，又考虑到了自身的信息

进一步实验：真正迭代多次看具体的结果

In [28]:
# 迭代5次，查看中间结果
num_layer = 5
H_wo = X
print('迭代0次（最初）\n')
print(H_wo)
for i in range(num_layer):
    H_wo = hat_a.dot(H_wo)
    print('第{}次\n'.format(i + 1))
    print(H_wo)

迭代0次（最初）

[[2 2 2]
 [1 1 1]
 [4 4 4]
 [3 3 3]]
第1次

[[ 6.  6.  6.]
 [ 6.  6.  6.]
 [ 7.  7.  7.]
 [10. 10. 10.]]
第2次

[[22. 22. 22.]
 [22. 22. 22.]
 [17. 17. 17.]
 [29. 29. 29.]]
第3次

[[73. 73. 73.]
 [73. 73. 73.]
 [46. 46. 46.]
 [90. 90. 90.]]
第4次

[[236. 236. 236.]
 [236. 236. 236.]
 [136. 136. 136.]
 [282. 282. 282.]]
第5次

[[754. 754. 754.]
 [754. 754. 754.]
 [418. 418. 418.]
 [890. 890. 890.]]


### 趋势与问题
趋势：随着迭代次数的增加，更新后的结点的感受野回越来越大，在这里则是更新后的结点由图中离自己相对更近的结点相加得到。 相应的， 更新后的结点特征值越来越大，和输入差距也越来越大，甚至随着迭代次数的增加，会溢出（越界）。其实，我们在做具体任务时只需要一个相对差距即可，原始数据和经过公平变化后的数据是几乎无差别的（归一化等）。

注意：这个简单的图，其实迭代两次，已经能获得整个图中的所有信息

解决办法：归一化

其中一种方式：除以结点的度

公式化：$$\mathbf{H}^{k + 1} = \mathbf{D}^{-1}\tilde{\mathbf{A}}\mathbf{H}^{k}$$

In [32]:
# 迭代5次，查看中间结果
num_layer = 5
H_w = X
print('迭代0次（最初）\n')
print(H_w)
D_1 = np.diag(1 / np.sum(A, axis=1)) 
print('D^-1\n')
print(D_1)
for i in range(num_layer):
    H_w = np.dot(D_1, hat_a.dot(H_w))
    print('第{}次\n'.format(i + 1))
    print(H_w)

迭代0次（最初）

[[2 2 2]
 [1 1 1]
 [4 4 4]
 [3 3 3]]
D^-1

[[0.5        0.         0.         0.        ]
 [0.         0.5        0.         0.        ]
 [0.         0.         1.         0.        ]
 [0.         0.         0.         0.33333333]]
第1次

[[3.         3.         3.        ]
 [3.         3.         3.        ]
 [7.         7.         7.        ]
 [3.33333333 3.33333333 3.33333333]]
第2次

[[ 4.66666667  4.66666667  4.66666667]
 [ 4.66666667  4.66666667  4.66666667]
 [10.33333333 10.33333333 10.33333333]
 [ 5.44444444  5.44444444  5.44444444]]
第3次

[[ 7.38888889  7.38888889  7.38888889]
 [ 7.38888889  7.38888889  7.38888889]
 [15.77777778 15.77777778 15.77777778]
 [ 8.37037037  8.37037037  8.37037037]]
第4次

[[11.57407407 11.57407407 11.57407407]
 [11.57407407 11.57407407 11.57407407]
 [24.14814815 24.14814815 24.14814815]
 [12.97530864 12.97530864 12.97530864]]
第5次

[[18.0617284  18.0617284  18.0617284 ]
 [18.0617284  18.0617284  18.0617284 ]
 [37.12345679 37.12345679 37.12345679]


### GCN
变化1：采用对称归一化

公式表达：
$$\mathbf{H}^{k + 1} = \mathbf{D}^{-1/2}\tilde{\mathbf{A}}\mathbf{D}^{-1/2}\mathbf{H}^{k}$$

变化2：引入可学习参数$\mathbf{W}$

公式表达：
$$\mathbf{H}^{k + 1} = \mathbf{D}^{-1/2}\tilde{\mathbf{A}}\mathbf{D}^{-1/2}\mathbf{H}^{k}\mathbf{W}^{k}$$

变化3：引入激活函数$\sigma$, 实现非线性

公式表达：
$$\mathbf{H}^{k + 1} = \sigma(\mathbf{D}^{-1/2}\tilde{\mathbf{A}}\mathbf{D}^{-1/2}\mathbf{H}^{k}\mathbf{W}^{k})$$

最终：一般为两层
公式表达：
$$\mathbf{Z} = f(\mathbf{X}, \mathbf{A}) = softmax(AReLu(\mathbf{A}\mathbf{X}\mathbf{W}^{(0)})\mathbf{W}^{(1)})$$

### 代码实现

In [11]:
# 加载数据并保存
import numpy as np
import scipy.sparse as sp
import torch

In [34]:
# 处理数据的相关函数
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot


def load_data(path="./data/cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)
    # my add
    #adj_temp1 = sparse_mx_to_torch_sparse_tensor(adj)
    # print(adj_temp1)
    print("未对称化之前的边数: ", adj.nnz)
    # my add end    
    
    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    
    # my add
    #adj_temp2 = sparse_mx_to_torch_sparse_tensor(adj)
    # print(adj_temp2)
    print("对称化之后的边数: ", adj.nnz)
    # my add end

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))
    
    # my add
    #adj_temp3 = sparse_mx_to_torch_sparse_tensor(adj)
    # print(adj_temp3)
    print("进一步标准化之后的边数: ", adj.nnz)
    # my add end

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)
    #print(adj)
    print("进一步转化为torch类型Coo格式后的边数: ", adj._nnz())

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test


def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [35]:
# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_data()
# save data
# torch.save({'adj': adj, 'features': features, 'labels': labels, 'idx_train': idx_train, 'idx_val': idx_val, 'idx_test': idx_test}, "cora.pth")


Loading cora dataset...
未对称化之前的边数:  5429
对称化之后的边数:  10556
进一步标准化之后的边数:  13264
进一步转化为torch类型Coo格式后的边数:  13264


In [2]:
# 加载数据
data = torch.load("cora.pth")

In [4]:
# 数据简单展示
# 邻接矩阵
adj = data["adj"]
print("The type of adj: ", type(adj))
# print(adj)
print("The size of adj: ", adj.size())
# 特征矩阵
features = data["features"]
print("The type of features: ", type(features))
print("The size of features: ", features.size())
# print(features)
# 标签
labels = data["labels"]
print("The type of labels: ", type(labels))
print("The size of labels: ", labels.size())
# print(labels)
# 训练集索引
idx_train = data["idx_train"]
print("The type of idx_train: ", type(idx_train))
print("The size of idx_train: ", idx_train.size())
# print(idx_train)
# 验证集索引
idx_val = data["idx_val"]
print("The type of idx_val: ", type(idx_val))
print("The size of idx_val: ", idx_val.size())
# print(idx_val)
# 测试集索引
idx_test = data["idx_test"]
print("The type of idx_test: ", type(idx_test))
print("The size of idx_test: ", idx_test.size())
# print(idx_test)

The type of adj:  <class 'torch.Tensor'>
The size of adj:  torch.Size([2708, 2708])
The type of features:  <class 'torch.Tensor'>
The size of features:  torch.Size([2708, 1433])
The type of labels:  <class 'torch.Tensor'>
The size of labels:  torch.Size([2708])
The type of idx_train:  <class 'torch.Tensor'>
The size of idx_train:  torch.Size([140])
The type of idx_val:  <class 'torch.Tensor'>
The size of idx_val:  torch.Size([300])
The type of idx_test:  <class 'torch.Tensor'>
The size of idx_test:  torch.Size([1000])


In [40]:
# 计算节点个数
nodes = features.shape[0]
print("The number of nodes: ", nodes)
# 计算每个特征的维数
d = features.shape[1]
print("The dimension of feature: ", d)
# 统计类别数
print("类别数为：", len(labels.unique()))
# 标签率，特指训练的标签占总数的多少
print("Label rate:", len(idx_train) / nodes)

The number of nodes:  2708
The dimension of feature:  1433
类别数为： 7
Label rate: 0.051698670605613


In [41]:
# 求精确度函数
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [42]:
import math
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

In [43]:
# GCN 卷积层
class GraphConvolution(Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
        
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
    
    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output
    
    def __repr__(self):
        return self.__class__.__name__ + ' (' \
                + str(self.in_features) + ' -> ' \
                + str(self.out_features) + ')'
    

In [44]:
import torch.nn as nn
import torch.nn.functional as F

In [45]:
class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()
        
        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout
        
    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

In [46]:
import time
# 训练模型
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()
    
    if not args.fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)
    
    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))
    

In [47]:
# 测试模块
def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss = {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))
    

In [59]:
# 主函数
# Parameters Setting
import argparse
import torch.optim as optim

parser = argparse.ArgumentParser()
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=200,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.01,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=16,
                    help='Number of hidden units.')
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')
# args = parser.parse_args()
args = parser.parse_args(args=[])
np.random.seed(args.seed)
args

Namespace(fastmode=False, seed=42, epochs=200, lr=0.01, weight_decay=0.0005, hidden=16, dropout=0.5)

In [60]:
# 定义模型与优化器
model = GCN(nfeat=features.shape[1],
            nhid=args.hidden,
            nclass=labels.max().item() + 1,
            dropout=args.dropout)
optimizer = optim.Adam(model.parameters(),
                       lr=args.lr, weight_decay=args.weight_decay)

In [61]:
# 训练模型
t_total = time.time()
for epoch in range(args.epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

Epoch: 0001 loss_train: 2.0450 acc_train: 0.0571 loss_val: 2.0439 acc_val: 0.0667 time: 0.0450s
Epoch: 0002 loss_train: 2.0174 acc_train: 0.0571 loss_val: 2.0204 acc_val: 0.0667 time: 0.0090s
Epoch: 0003 loss_train: 1.9961 acc_train: 0.0643 loss_val: 1.9987 acc_val: 0.0667 time: 0.0080s
Epoch: 0004 loss_train: 1.9707 acc_train: 0.0714 loss_val: 1.9783 acc_val: 0.0700 time: 0.0070s
Epoch: 0005 loss_train: 1.9495 acc_train: 0.0929 loss_val: 1.9593 acc_val: 0.1733 time: 0.0070s
Epoch: 0006 loss_train: 1.9370 acc_train: 0.1786 loss_val: 1.9415 acc_val: 0.1567 time: 0.0070s
Epoch: 0007 loss_train: 1.9120 acc_train: 0.2000 loss_val: 1.9247 acc_val: 0.1567 time: 0.0080s
Epoch: 0008 loss_train: 1.8935 acc_train: 0.2000 loss_val: 1.9089 acc_val: 0.1567 time: 0.0080s
Epoch: 0009 loss_train: 1.8898 acc_train: 0.2000 loss_val: 1.8946 acc_val: 0.1567 time: 0.0080s
Epoch: 0010 loss_train: 1.8594 acc_train: 0.1929 loss_val: 1.8812 acc_val: 0.1567 time: 0.0080s
Epoch: 0011 loss_train: 1.8569 acc_train

Epoch: 0171 loss_train: 0.4548 acc_train: 0.9429 loss_val: 0.7283 acc_val: 0.8267 time: 0.0070s
Epoch: 0172 loss_train: 0.4575 acc_train: 0.9571 loss_val: 0.7270 acc_val: 0.8200 time: 0.0080s
Epoch: 0173 loss_train: 0.4730 acc_train: 0.9143 loss_val: 0.7260 acc_val: 0.8200 time: 0.0080s
Epoch: 0174 loss_train: 0.4659 acc_train: 0.9500 loss_val: 0.7259 acc_val: 0.8133 time: 0.0070s
Epoch: 0175 loss_train: 0.4237 acc_train: 0.9214 loss_val: 0.7254 acc_val: 0.8167 time: 0.0070s
Epoch: 0176 loss_train: 0.4188 acc_train: 0.9500 loss_val: 0.7248 acc_val: 0.8167 time: 0.0070s
Epoch: 0177 loss_train: 0.4624 acc_train: 0.9214 loss_val: 0.7240 acc_val: 0.8100 time: 0.0070s
Epoch: 0178 loss_train: 0.4730 acc_train: 0.9357 loss_val: 0.7235 acc_val: 0.8100 time: 0.0080s
Epoch: 0179 loss_train: 0.4530 acc_train: 0.9571 loss_val: 0.7209 acc_val: 0.8100 time: 0.0070s
Epoch: 0180 loss_train: 0.3857 acc_train: 0.9643 loss_val: 0.7190 acc_val: 0.8133 time: 0.0070s
Epoch: 0181 loss_train: 0.4440 acc_train

In [62]:
# 测试模型
test()

Test set results: loss = 0.7219 accuracy= 0.8320
